# Introduction to CVXPY for Optimization

This notebook is from 2024 by Zikai Xiong and Sean Lo. Revised in 2025 by Karl Zhu.

CVXPY is an open-source Python-embedded modeling language for convex optimization problems. It lets you express your problem in a natural way that follows the math, rather than in the restrictive standard form required by solvers.

If you really hate using Julia, then CVXPY is the alternative in Python that can help you out. Other popular Python packages include PuLP (for MILPs) and Pyomo (linear, nonlinear, convex, mixed-integer, stochastic, and more...).

To make things less repetitive and more interesting, we will also use PDLP - a new method to solve LPs that can be extended to be solved using GPUs. (https://arxiv.org/abs/2311.12180)

The documentation for:
- CVXPY https://www.cvxpy.org/
- PuLP https://coin-or.github.io/pulp/
- Pyomo https://www.pyomo.org/

In [1]:
# Step 1: Install necessary packages
#!pip install cvxpy
#!pip install numpy
#!pip install gurobipy
#!pip install ortools==9.7.2996 # (The version after 9.8 is not supported for cvxpy)
#!pip install "cvxpy[PDLP]"


In [2]:
# Step 2: import cvxpy and numpy packages
import cvxpy as cp
import numpy as np

In [ ]:
# Step 3: check installed solvers
print(cp.installed_solvers())

## Section 1: Linear Optimization Problem

Define the optimization variables

In [4]:
x = cp.Variable()
y = cp.Variable()

Define the objective function and constraints

In [5]:
objective = cp.Maximize(3*x + 2*y)
constraints = [2*x + y <= 10, x + 3*y <= 12, x >= 0, y >= 0]

Create the optimization problem

In [6]:
linear_problem = cp.Problem(objective, constraints)

Solve the problem (We use the solver Gurobi but you can also use other solvers. )

In [ ]:
linear_problem.solve(solver=cp.GUROBI,verbose=True)

Display the results

In [ ]:
print("Optimal value:", linear_problem.value)
print("Optimal x:", x.value)
print("Optimal y:", y.value)

## Section 2: A First-Order LP Solver (PDLP)

In this section we solve a randomly generated "big" LP instance using cvxpy and different solvers.

The problem is 
$$
\min \ c^\top x, \\
\text{s.t.} \ Ax \le b \ ,
$$
in which $A$ has $m$ rows and $n$ columns. 

The data is created as follows:

In [9]:
np.random.seed(123)
m = 5000
n = 3000

# Generate A, b, c so that (x0,y0) is an optimal primal-dual solution to the LP problem 
s0 = np.random.randn(m)
y0 = np.maximum(-s0, 0)
s0 = np.maximum(s0, 0)
x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A @ x0 + s0
c = -A.T @ y0

# Define the LP using CVXPY
#### YOUR CODE HERE

Solve the problem using the primal simplex method of GUROBI (about 700 seconds).

In [ ]:
opt_val=big_LP.solve(solver=cp.GUROBI, verbose=True, Method=0)
solution=x.value
print(opt_val)
print(solution)

Solve the problem using the dual simplex method of GUROBI (about 800 seconds.)

In [ ]:
# Solve the problem using the default GUROBI dual simplex method
opt_val=big_LP.solve(solver=cp.GUROBI, verbose=True, Method=1)

Solve the problem using PDLP (a first-order method for LP).  

The PDLP is the primal-dual hybrid gradient (PDHG) method with some practical enhancements. 
See more details of the mathematical background in https://developers.google.com/optimization/lp/pdlp_math. 

In [ ]:
opt_val=big_LP.solve(solver=cp.PDLP, verbose=True)

It needs only 100 seconds!!!